<a href="https://colab.research.google.com/github/Kashirajaji/pipelines/blob/main/Cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cross-Validation

Como vimos recientemente en la teoria , podemos ver que es necesario evaluar el tema de overfitting y underfitting

Como vimos recien el tema del tradeoff entre bias y variance , y gracias a eso podemos observar cuando uno de nuestros modelos esta sufriendo over o under fitting. 

Para poder ayudar a mejor y no obtener overfittin uno de los metodos mas usados y generalmente apreciado a usar casi siempre es poder hacer CrossValidation dentro del conjunto train

Aqui vamso a evaluar varias formas de poder trabajarlo

## Breve recordatorio

Para poder definir que nuestro algoritmo sufre de algunos de estos comportamientos , nosotros evaluamos dos metricas que son el Bias y el Variance

BIAS : es un tipo de error que indica la diferencia que existe entre la predicción del modelo y el valor actual

VARIANCE : Este error lo podemos entender como que tan sensible es nuestro modelo a los datos,  cambiando y evlaunado diferentes tipos de datos a validar vemos como cambia su perfromance 

Ejemplo de lo que puede suceder

- ***Ejemplo 1: Overfitting, low bias and high variance***

    - Training error = 1%

    - Validation error = 11%

Este algoritmo tiene un low bias (1%) pero sufre de high variance (10%). El clasificador tiene un bajo training error, pero falla al generalizar sobre el validation set.

- ***Ejemplo 2: Underfitting, high bias and low variance***

    - Training error = 15%

    - Validation error = 16%

Aquí el bias es 15% y el variance es 1%. El clasificador se ajusta pobremente sobre el conjunto de entrenamiento con un 15% de error (underfitting), pero el error sobre el validation set es sólo un poco más alto que éste.

- ***Ejemplo 3: High bias and high variance***

    - Training error = 15%

    - Validation error = 30%

Aquí el bias es 15%, y el variance es 15%. Este clasificador presenta un bajo rendimiento sobre el conjunto de entrenamiento (high bias), y una performance aún peor sobre el validation set (high variance).

- ***Ejemplo 4: Buen clasificador, low bias and low variance***

    - Training error = 0,5%

    - Validation error = 1%

Este es  un ejemplo de un buen clasificador, con low bias y low variance.



En base a esta info tenemos un prevencion a tomar en cuenta

Formas de prevenir el underfitting:

- Tratar los datos correctamente, eliminando outliers y variables innecesarias.
- Utilizar modelos más complejos.
- Ajustar los parámetros de nuestros modelos.
- Aumentar las iteraciones en los algoritmos iterativos

Formas de prevenir el overfitting:

- Dividir nuestros datos en training, validación y testing.
- Obtener un mayor número de datos.
- Ajustar los parámetros de nuestros modelos.
- Utilizar modelos más simples.
- Los datos vienen de distintas distribuciones.
- Bajar el número de iteraciones en los algoritmos iterativos.

![flujo](https://intermediait.com/wp-content/uploads/2019/12/Flujo-de-trabajo-para-reducir-el-BIAS-y-VARIANCE.png)


##ejemplo de trabajo con Bias y Variance

Vamos a trabjar en el conjunto de titanic para evaluar como podemos trabajar sobre esto

In [ ]:
#Manejo de datos de Titanic

import pandas as pd
from sklearn.preprocessing import MinMaxScaler , OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier


In [ ]:
#train
df_train = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

#test
df_test = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/test.csv')

#target test
df_submission = pd.read_csv('my_submission.csv')

df_train.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [ ]:
#sabemos que tiene datos nulos y vamos hacer un scaler de los datos 

#split target and data
X_train = df_train.drop('Survived', axis=1)
y_train = df_train['Survived']

#delete all columns not necessary
X_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)



In [ ]:
X_train.select_dtypes(include='O').columns.tolist()

['Sex', 'Embarked']

In [ ]:
#obtener columnas categoricas
cat_col = X_train.select_dtypes(include='O').columns.tolist()


#generar a pipeline para numericas 
numeric_transformer = Pipeline(
    steps=[('fill', SimpleImputer(strategy='median')), 
           ("scaler", MinMaxScaler())])

#generar un pipeline para categoricas
categorical_transformer = Pipeline(steps=[('fill_cat', SimpleImputer(strategy='most_frequent')),
                                          ("ohe",OneHotEncoder(drop='first', handle_unknown="ignore"))])


#funcion que realiza el trabajo para cada pipeline y luego unirlo
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, make_column_selector(dtype_exclude=object)),
        ("cat", categorical_transformer, make_column_selector(dtype_include = object)),#['Sex', 'Embarked']
    ]
)

#obtencio. de los procesos en el Xtrain
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('fill',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', MinMaxScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f80134c5350>),
                                ('cat',
                                 Pipeline(steps=[('fill_cat',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f80134c5e90>)])

In [ ]:
preprocessor.named_transformers_['cat']['ohe'].get_feature_names_out(cat_col)

array(['Sex_male', 'Embarked_Q', 'Embarked_S'], dtype=object)

In [ ]:
#trasnformacion de X_train 
X_array_enc = preprocessor.transform(X_train)
#genero nombres de columnas
columns_enc = np.append(X_train.select_dtypes(exclude='object').columns, preprocessor.named_transformers_['cat']['ohe'].get_feature_names_out(cat_col))
X_train_enc = pd.DataFrame(X_array_enc, columns = columns_enc)
X_train_enc

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1.0,0.271174,0.125,0.000000,0.014151,1.0,0.0,1.0
1,0.0,0.472229,0.125,0.000000,0.139136,0.0,0.0,0.0
2,1.0,0.321438,0.000,0.000000,0.015469,0.0,0.0,1.0
3,0.0,0.434531,0.125,0.000000,0.103644,0.0,0.0,1.0
4,1.0,0.434531,0.000,0.000000,0.015713,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...
886,0.5,0.334004,0.000,0.000000,0.025374,1.0,0.0,1.0
887,0.0,0.233476,0.000,0.000000,0.058556,0.0,0.0,1.0
888,1.0,0.346569,0.125,0.333333,0.045771,0.0,0.0,1.0
889,0.0,0.321438,0.000,0.000000,0.058556,1.0,0.0,0.0


In [ ]:
#lo mismo para test


X_test = df_test
y_test = df_submission['Survived']

X_test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],axis=1, inplace=True)

X_array_enc_test = preprocessor.transform(X_test)

#genero nombres de columnas
columns_enc = np.append(X_test.select_dtypes(exclude='object').columns, preprocessor.named_transformers_['cat']['ohe'].get_feature_names_out(cat_col))
X_test_enc = pd.DataFrame(X_array_enc_test, columns = columns_enc)
X_test_enc

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1.0,0.428248,0.000,0.000000,0.015282,1.0,1.0,0.0
1,1.0,0.585323,0.125,0.000000,0.013663,0.0,0.0,1.0
2,0.5,0.773813,0.000,0.000000,0.018909,1.0,1.0,0.0
3,1.0,0.334004,0.000,0.000000,0.016908,1.0,0.0,1.0
4,1.0,0.271174,0.125,0.166667,0.023984,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
413,1.0,0.346569,0.000,0.000000,0.015713,1.0,0.0,1.0
414,0.0,0.484795,0.000,0.000000,0.212559,0.0,0.0,0.0
415,1.0,0.478512,0.000,0.000000,0.014151,1.0,0.0,1.0
416,1.0,0.346569,0.000,0.000000,0.015713,1.0,0.0,1.0


In [ ]:
#verificar nombre de columnas

set(X_train_enc.columns) - set(X_test_enc.columns) 

set()

In [ ]:
set(X_train_enc.columns) - set(['Age', 'Fare'])

{'Embarked_Q', 'Embarked_S', 'Parch', 'Pclass', 'Sex_male', 'SibSp'}

En base a estos valores ya trasnformado vamos a trabajar como generalmente lo hacemos.

Lo primero y mas importante es que debemos tener un split entre el Train y el test, dado que nosotros trabajaremos sobre el Train.  
Alli luego tendremos varias opciones a poder decidir.

 - Holdout validate
 - Kfold
 - LeaveOut

 Y tenemos una opcion dentro del Kfold donde podemos seleccionar que la separacion sea estratificada 


Lo mas imporatante ***TRABAJAMOS SOBRE EL X_TRAIN***

### HOLDOUT

El holdout es un metodo sencillo donde dentro del dataset del train splitemos una parte que denominaremos data_validate, que sera de manera aleatoria una seleccion de un a parte para poder comparar

- Ventajas
    - Metodo sencillo para poder validar , 

- Desventajas
    - Genera un alto ruido o alto variabilidad en la seleccion de ese dataset para validar ,  puede ser muy diferente a la realidad ese conjunto (problema de varianza)
    - Menos informaicion para entrenar

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


X_train , X_validate, y_train_hold, y_validate = train_test_split(X_train_enc, y_train , test_size=0.1)

model = DecisionTreeClassifier(max_depth=1, max_features=2,random_state=7)

model.fit(X_train, y_train_hold)

pre = model.predict(X_validate)
accuracy_score(y_validate, pre)

0.6222222222222222

## Kfold

Primero que veremos es el K-Fold donde tiene varias particualridades para evaluar donde vamos a ver que tanto nos ayuda a poder mejorar 

![idea](https://www.aprendemachinelearning.com/wp-content/uploads/2020/02/kfolds_animated.gif)

Esto esun proceso iterativo donde va seleccionado parte del datasets en secciones
Alli se realiza su entrenamiento y va scoreando cada subconjunto para ver como performa

- Ventajas
    - Computo de procesamiento, daod que se puede interar en pocas secciones rapidamente. 

    - Ayuda a [poder balancear mejor el bias y varianza , es decir la poder realizar varios entrenamiento en todo el conjunto y testeando su parte desconocida , ayuda a evaluar si todo el conjunto con la relacion del algoritmo tiene una tendencia mala es su predeccion como tambien en la parte de prediccion en conjunto desconocidos

- Desventajas

    - No se debeusar para datasets muy grandes , alli solo se ultiliza el holdout dado que la cantidad de datos que van a tener para validar ya alcanza para tener una buena muestra 

In [ ]:
#metodo para crossvalidation
CV_scores = cross_val_score(model, X_train_enc, y_train, cv=10)


In [ ]:
#lista de todos los entrenamiento de kfold
CV_scores

array([0.5       , 0.57303371, 0.53932584, 0.61797753, 0.61797753,
       0.66292135, 0.62921348, 0.61797753, 0.62921348, 0.60674157])

In [ ]:
print("Cross validation score es  %.5f ± %0.2f" % (CV_scores.mean(), CV_scores.std()))

Cross validation score es  0.59944 ± 0.05


In [ ]:
#ejemplo para estratificado

kfold = StratifiedKFold(n_splits=10,shuffle=True)

model = DecisionTreeClassifier(max_depth=1 , max_features=2 ,random_state=7)
CV_scores = cross_val_score(model, X_train_enc, y_train, cv=kfold)

print("Cross validation score es  %.5f ± %0.2f" % (CV_scores.mean(), CV_scores.std()))

Cross validation score es  0.60381 ± 0.05


##Ejemplo donde determinamos su underfitting o overfitting

In [ ]:
!pip install colorama
from colorama import Fore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

def print_scores(model ,X_train , y_train, predictions , cv_splites=10):
    
    print(Fore.BLUE , "El promedion de score en el train es  %.5f" % model.score(X_train, y_train))

    #cross validation

    CV_scores = cross_val_score(model, X_train, y_train, cv=cv_splites)
    
    print(Fore.BLACK ,"Los scores del cross validation son: \n",CV_scores)
    print(Fore.BLACK ,"El score minimo es %.3f" % min(CV_scores))
    print(Fore.BLACK ,"El maximo score es %.3f" % max(CV_scores))
    print(Fore.YELLOW ,"Cross validation score es  %.5f ± %0.2f" % (CV_scores.mean(), CV_scores.std()))
    print(Fore.RED ,"El test score es  %.5f " % accuracy_score(y_test,predictions))

#### ejemplo

Veamos que conclusiones sacamos

In [ ]:
model = DecisionTreeClassifier(max_depth=1 , max_features=2 ,random_state=7)
model.fit(X_train_enc, y_train)

predictions = model.predict(X_test_enc)
print_scores(model, X_train_enc, y_train, predictions)

 El promedion de score en el train es  0.61616
 Los scores del cross validation son: 
 [0.5        0.57303371 0.53932584 0.61797753 0.61797753 0.66292135
 0.62921348 0.61797753 0.62921348 0.60674157]
 El score minimo es 0.500
 El maximo score es 0.663
 Cross validation score es  0.59944 ± 0.05
 El test score es  0.63636 


###conclusiones

Tengo un modelo underfitting 

porque alto bias(0.61) y low variance (0.61-0.599)


In [ ]:
model = DecisionTreeClassifier(random_state=7)
model.fit(X_train_enc, y_train)

predictions = model.predict(X_test_enc)
print_scores(model, X_train_enc, y_train, predictions)

 El promedion de score en el train es  0.97980
 Los scores del cross validation son: 
 [0.71111111 0.78651685 0.74157303 0.7752809  0.85393258 0.7752809
 0.80898876 0.74157303 0.82022472 0.78651685]
 El score minimo es 0.711
 El maximo score es 0.854
 Cross validation score es  0.78010 ± 0.04
 El test score es  0.80622 


###conclusiones

low biaS ( 0.97)
high variance (0.97 - 0.78) = 0.l9 

OVERFITTING


In [ ]:
model = DecisionTreeClassifier(max_depth=3,  max_features= 4, random_state=7)
model.fit(X_train_enc, y_train)

predictions = model.predict(X_test_enc)
print_scores(model, X_train_enc, y_train, predictions)

 El promedion de score en el train es  0.78676
 Los scores del cross validation son: 
 [0.74444444 0.75280899 0.75280899 0.83146067 0.82022472 0.79775281
 0.80898876 0.75280899 0.82022472 0.78651685]
 El score minimo es 0.744
 El maximo score es 0.831
 Cross validation score es  0.78680 ± 0.03
 El test score es  0.82775 


###Conclusiones




### LeaveOneOut

Es el mismo proceso iterativo , solo que ahora en vez de hacerlo en conjuntos de split , lo que hace es entrenar con casi todo el x_train pero excluyendo una observacion donde se hara la prediccion

Se realiza el promedio de todas la iteracciones

Con eso itera sobre todo el conjunto

- Ventajas
    - Al reviasr una a una las predicciones eso hace que No sufra la varianza , dado que es mas exacto en cada iteracion
    - Su performance es mas exacta porque toma todo el conjunto de datos casi y por ende no tiende a sobreestimar

- Desventaja
    - Demasiado computo de proceso , lo cual puede ser interminable

![loocv](https://i1.wp.com/dataaspirant.com/wp-content/uploads/2020/12/7-LOOCV-Leave-One-Out-Cross-Validation.png?resize=300%2C225&ssl=1)



In [ ]:
#conclusiones 
from sklearn.model_selection import LeaveOneOut
cv = LeaveOneOut()

#build multiple linear regression model
model = DecisionTreeClassifier(max_depth=1 , max_features=2 ,random_state=7)

#use LOOCV to evaluate model
scores = cross_val_score(model, X_train_enc, y_train,
                         cv=cv, n_jobs=-1)

In [ ]:
np.mean(scores)

# Grid Search con CV

In [ ]:
##Gridsearch con CV

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


# parametros a correr
param_grid = {
    'max_depth': [3, 4, 5],
    'max_features': [3,4,6],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(X_train_enc, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5], 'max_features': [3, 4, 6],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2)

In [ ]:
grid_search.best_score_

0.8260381593714928

In [ ]:
grid_search.best_params_

{'max_depth': 5, 'max_features': 4, 'n_estimators': 200}

In [ ]:
pd.DataFrame(grid_search.cv_results_).sort_values('rank_test_score', ascending=True).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
29,0.477740,0.004584,0.045108,0.000373,5,4,200,"{'max_depth': 5, 'max_features': 4, 'n_estimat...",0.818182,0.838384,0.821549,0.826038,0.008837,1
30,0.724651,0.012581,0.067000,0.002540,5,4,300,"{'max_depth': 5, 'max_features': 4, 'n_estimat...",0.804714,0.841751,0.828283,0.824916,0.015307,2
24,0.233483,0.004616,0.022669,0.001836,5,3,100,"{'max_depth': 5, 'max_features': 3, 'n_estimat...",0.808081,0.838384,0.828283,0.824916,0.012598,2
20,0.249325,0.010198,0.024824,0.001005,4,6,100,"{'max_depth': 4, 'max_features': 6, 'n_estimat...",0.814815,0.831650,0.828283,0.824916,0.007274,2
19,2.325997,0.006963,0.208972,0.002529,4,4,1000,"{'max_depth': 4, 'max_features': 4, 'n_estimat...",0.814815,0.838384,0.821549,0.824916,0.009912,2
